# Super Bowl LIX Analysis: Seahawks vs. Patriots

**Goal**: Deep Dive Matchup Analysis.
1. **Roster Strength**: Global Efficiency & Star Power.
2. **Positional Matchups**: Head-to-head comparison of position groups (Offense & Defense).
3. **X-Factors**: Key players with surplus value.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

# Load Data
df = pd.read_csv('../data/processed/nfl_master_table.csv')

# Filter for 2025 Season (Contracts) & 2024 Performance
sb_teams = ['SEA', 'NE']

# 1. Get 2025 Cost Structure
cap_2025 = df[(df['year'] == 2025) & (df['team'].isin(sb_teams))][['clean_name', 'player_name', 'team', 'position', 'age', 'cap_hit_m']].copy()

# 2. Get 2024 Production (Baseline)
perf_2024 = df[(df['year'] == 2024)][['clean_name', 'AV_Proxy']].copy()
perf_2024.rename(columns={'AV_Proxy': 'AV_2024'}, inplace=True)

# 3. Merge
sb_df = pd.merge(cap_2025, perf_2024, on='clean_name', how='left')
sb_df['AV_2024'] = sb_df['AV_2024'].fillna(0)
sb_df['efficiency'] = sb_df['AV_2024'] / sb_df['cap_hit_m']
sb_df.replace([np.inf, -np.inf], 0, inplace=True)

print(f"Analyzed {len(sb_df)} contracts for SEA/NE.")

Analyzed 131 contracts for SEA/NE.


## 1. Roster Strength: Star Power
Who has the higher ceiling? (Sum of Top 10 Players' AV)

In [2]:
top_10 = sb_df.groupby('team').apply(lambda x: x.nlargest(10, 'AV_2024')['AV_2024'].sum()).reset_index()
top_10.columns = ['team', 'Top10_AV_Sum']
print("Top 10 Players Total AV (Star Power):")
print(top_10)

# Depth (Players 11-30)
depth = sb_df.groupby('team').apply(lambda x: x.sort_values('AV_2024', ascending=False).iloc[10:30]['AV_2024'].sum()).reset_index()
depth.columns = ['team', 'Depth_AV_Sum']
print("\nRoster Depth (Players 11-30 Total AV):")
print(depth)

Top 10 Players Total AV (Star Power):
  team  Top10_AV_Sum
0   NE       40.4328
1  SEA       57.5008

Roster Depth (Players 11-30 Total AV):
  team  Depth_AV_Sum
0   NE       32.5008
1  SEA       29.0376


/var/folders/2c/hn46snbj3qb84rkyzmls72240000gn/T/ipykernel_40246/363664580.py:1: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  top_10 = sb_df.groupby('team').apply(lambda x: x.nlargest(10, 'AV_2024')['AV_2024'].sum()).reset_index()
/var/folders/2c/hn46snbj3qb84rkyzmls72240000gn/T/ipykernel_40246/363664580.py:7: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  depth = sb_df.groupby('team').apply(lambda x: x.sort_values('

## 2. Positional Matchups
Grouping positions to compare unit strength.

In [3]:
def map_pos_group(pos):
    if pd.isna(pos): return 'Other'
    pos = pos.upper()
    if pos in ['QB']: return 'QB'
    if pos in ['RB', 'WR', 'TE', 'FB']: return 'Skill Players'
    if pos in ['LT', 'RT', 'C', 'G', 'LG', 'RG', 'T', 'OL']: return 'OL'
    if pos in ['DE', 'DT', 'ILB', 'OLB', 'LB', 'ED', 'DL', 'NT']: return 'Front 7'
    if pos in ['CB', 'S', 'SS', 'FS', 'DB']: return 'Secondary'
    if pos in ['K', 'P', 'LS']: return 'Special Teams'
    return 'Other'

sb_df['Pos_Group'] = sb_df['position'].apply(map_pos_group)

# Aggregated AV by Group
pos_matchup = sb_df.groupby(['team', 'Pos_Group'])['AV_2024'].sum().unstack(0)
pos_matchup['Advantage'] = pos_matchup.apply(lambda row: 'SEA' if row.get('SEA', 0) > row.get('NE', 0) else 'NE', axis=1)
pos_matchup['Diff'] = abs(pos_matchup['SEA'] - pos_matchup['NE'])

print("POSITIONAL MATCHUP (Total 2024 AV):")
print(pos_matchup[['SEA', 'NE', 'Advantage', 'Diff']].sort_values('Diff', ascending=False))

POSITIONAL MATCHUP (Total 2024 AV):
team               SEA       NE Advantage     Diff
Pos_Group                                         
Other          29.3000  17.9400       SEA  11.3600
QB             14.6664   8.4808       SEA   6.1856
Front 7         3.0600   8.7800        NE   5.7200
Secondary      14.1000  13.2920       SEA   0.8080
OL              0.0000   0.2560        NE   0.2560
Skill Players  25.7280  25.5728       SEA   0.1552
Special Teams   0.0800   0.0000       SEA   0.0800


## 3. Notable X-Factors (Surplus Value)
Players outperforming their 2025 Cap Hit.

In [4]:
sb_df['Surplus'] = sb_df['AV_2024'] - (sb_df['cap_hit_m'] / 2.0) # Approx 1 AV = $2M assumption

for team in sb_teams:
    print(f"\n--- {team} Best Value (X-Factors) ---")
    print(sb_df[sb_df['team'] == team].sort_values('Surplus', ascending=False).head(3)[['player_name', 'position', 'age', 'cap_hit_m', 'AV_2024']])


--- SEA Best Value (X-Factors) ---
           player_name position  age  cap_hit_m  AV_2024
53     Zach Charbonnet       RB  NaN   1.884114   5.7840
2          Sam Darnold       QB  NaN  14.400000  11.7208
46  Kenneth Walker III       RB  NaN   2.693517   5.1720

--- NE Best Value (X-Factors) ---
            player_name position  age  cap_hit_m  AV_2024
75      Demario Douglas       WR  NaN   1.073413    3.112
19  Rhamondre Stevenson       RB  NaN   5.543696    5.264
13           Drake Maye       QB  NaN   8.336984    6.628
